In [1]:
!pip install beautifulsoup4

In [4]:
!pip install html5lib
!pip install PyQt5

     ---------------------------------------- 6.8/6.8 MB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 50.1/50.1 MB 3.1 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 5] \u041e\u0442\u043a\u0430\u0437\u0430\u043d\u043e \u0432 \u0434\u043e\u0441\u0442\u0443\u043f\u0435: 'C:\\Users\\egorf\\anaconda3\\Lib\\site-packages\\PyQt5\\QtCore.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
import json
from bs4 import BeautifulSoup
import urllib.request
import re
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEnginePage
import sys

class Client(QWebEnginePage):
    
    def __init__(self, url):
        self.app = QApplication(sys.argv)
        QWebEnginePage.__init__(self)
        self.loadFinished.connect(self.on_page_load)
        self.setUrl(QUrl(url))
        self.app.exec_()
        
    def on_page_load(self):
        self.app.quit()

def dl_vk_data(group_id, parse, offset=0, count=5, use_dryscrape=False):
    with urllib.request.urlopen(f'https://api.vk.com/method/wall.get?access_token=247f3d70247f3d70247f3d70d6276dfd5c2247f247f3d70478c4cc7443965779421d6d7&owner_id={group_id}&v=5.131&offset={offset}&count={count}') as f:
        json_string = f.read().decode('utf-8')
    json_res = json.loads(json_string)['response']
    for item in json_res['items']:
        print("-" * 50)
        text = item['text']
        print(text)
        for attachment in item['attachments']:
            link = attachment.get('link')
            if not link:
                continue
            url = link['url']
            print(url)
            if use_dryscrape:
                Client(url).toHtml(lambda arg: print(arg))
            else:
                with urllib.request.urlopen(url) as f:
                    html = f.read().decode('utf-8')
                parse(html)

In [5]:
def parse_rbc(html):
    soup = BeautifulSoup(html)
            
    title_html = soup.find("div", class_="article__header__title")
    if title_html:
        title = str(title_html.h1.string).strip()
        print(title)

    article_html = soup.find("div", class_="article__text")
    if article_html:
        paragraphs = article_html.find_all(re.compile("(p|h|li)"))
        text = ""
        for block in paragraphs:
            for s in block.stripped_strings:
                text += str(s) + " "
        print(text)

    tags_html = soup.find("div", class_="article__tags__container")
    if tags_html:
        tags = []
        for tag in map(lambda s: str(s.string), tags_html.find_all("a")):
            tags.append(tag)
        print(tags)
            
dl_vk_data(-25232578, parse_rbc)

--------------------------------------------------
Главные новости РБК в прямом эфире. На канале представлены главные новости политики, экономики и бизнеса России, комментарии аналитиков, финансовые данные с российских и мировых биржевых систем. 
Эксклюзивные интервью с ведущими политиками и бизнесменами; Аналитические обзоры, прогнозы и комментарии экспертов. 
 
Смотрите нас в RUTUBE: https://rutube.ru/channel/24141691/ 
Смотрите нас в VK: https://vk.com/rbc 
Смотрите нас в Яндекс.Дзен: https://zen.yandex.ru/rbc.ru
--------------------------------------------------
Союз российских пивоваров считает, что транснациональные компании в России могут применять высокоактивные ферментные препараты и катализаторы для ускоренного пивоварения
https://www.rbc.ru/business/21/02/2023/63f462db9a79473f2d6af43f?utm_source=vk&utm_medium=social&utm_campaign=preview
Российские пивовары попросили разобраться с варкой пива за три дня
Союз российских пивоваров считает, что транснациональные компании в Росси

In [6]:
def parse_ria(html):
    soup = BeautifulSoup(html)
    title_html = soup.find("div", class_="article__title")
    second_title_html = soup.find("h1", class_="article__second-title")
    if title_html and second_title_html:
        title = str(title_html.string) + ". " + str(second_title_html.string)
        print(title)
        
    article_html = soup.find_all("div", class_="article__text")
    if article_html:
        article = ""
        for block in article_html:
            for s in block.stripped_strings:
                article += str(s) + " "
        print(article)
        
    tags_html = soup.find_all("a", class_="article__tags-item")
    if tags_html:
        tags = []
        for tag in tags_html:
            tags.append(str(tag.string))
        print(tags)
    
dl_vk_data(-15755094, parse_ria)

--------------------------------------------------
Волонтеры сообщества [club212156263|"Неравнодушные"] помогают людям в зоне СВО. Они не только привозят гумпомощь пострадавшим, но и оказывают психологическую поддержку. Активисты рассказывают, что для таких людей самое главное - выговориться и быть услышанным. Для многих волонтеров те, кому они помогают, стали уже настоящей семьей. А главная цель движения "Неравнодушные" - дать понять тем, кто нуждается в поддержке, что они не одиноки в своей беде, что за ними целая страна и огромный тыл.
--------------------------------------------------
США и Евросоюз так и не определились, чем, с их точки зрения, должны закончиться боевые действия на Украине. В НАТО нет единства по этому вопросу. Кто на Западе наиболее агрессивно настроен по отношению к Москве и с чем это связано — в нашем материале.
https://ria.ru/20230221/ukraina-1853207020.html
МОСКВА, 21 фев — РИА Новости, Михаил Катков. США и Евросоюз так и не определились, чем, с их точки зрен